**Cuatro en Raya**

In [6]:
import numpy as np


class Board():
    def __init__(self):
        self.state = np.zeros((4,4))
# determina si la celda en la que se va a ocupar ya esta ocupada previamente
    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]
#Inserta el simbolo en la tabla
    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError ("movimiento ilegal !")
# definimos la comprobacion del tablero, se verifican las lineas, columnas y diagonales y el empate
    def is_game_over(self):
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1:
            return 1
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:
            return -1
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),
            sum([self.state[i, 4 - i - 1] for i in range(4)]),
        ]
        if diag_sums[0] == 4 or diag_sums[1] == 4:
            return 1
        if diag_sums[0] == -4 or diag_sums[1] == -4:
            return -1
        if len(self.valid_moves()) == 0:
            return 0
        return None
#'Limpia el tablero para iniciar de nuevo'
    def reset(self):
        self.state = np.zeros((4,4))


In [7]:
from tqdm import tqdm

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

#'juega' o entrena el juego la cantidad self de veces, actualiza la cantidad de victorias y de parte de quien
    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins

#Asigna la recompensa por cada victoria en todas las iteraciones al player ganador
    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:
            for player in self.players:
                player.reward(0.5)
        else:
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [8]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}
        #tasa de aprendizaje
        self.alpha = alpha
        #vector de las posiciones de la partida
        self.positions = []
        #probabilidad de que se haga una exploracion
        self.prob_exp = prob_exp

    def reset(self):
        self.positions = []
#
    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        #si el valor es inferior a la probabilidad de exploracion, realizara una
        #exploracion, realizando un movimiento aleatorio
        if explore and np.random.uniform(0, 1) < self.prob_exp:

            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        #explota el sistema, utilizando las posiciones con mejores valores en cuanto a estrategia o es decir victorias conseguidas
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))


    def reward(self, reward):
        #Se actualiza el valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

In [10]:
#se crean dos jugadores y al primero o explorador se le da una probabilidad de exploracion,
# para que sea el que descubra nuevas posiciones con valores y force la actualizacion
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()
#Se envian los dos jugadores a jugar
game = Game(agent1, agent2)
#la cantidad de veces que se jugara o iterara
game.selfplay(30000)

100%|██████████| 30000/30000 [10:05<00:00, 49.58it/s]


[10199, 8165]

In [11]:
import pandas as pd
#Ordena los vectores de todas las posiciones de las jugadas, de acuerdo al estado en el que termino el juego
#0 representa que la posicion estuvo vacia y -1 y 1 para posiciones ocupadas por simbolos de uno u otro jugador
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0. -1. 0. 1. 0. 0. 0. 1. 0. -1. 0. ...,1.0
1,[ 0. 1. 0. -1. 0. 1. 0. 0. 0. 1. 0. -...,1.0
2,[ 0. 0. 1. 0. 0. 0. 1. -1. 0. 0. 1. -...,1.0
3,[ 0. 1. 0. 0. 0. 1. 0. -1. 0. 1. 0. -...,1.0
4,[ 0. 0. 0. 1. 0. -1. 0. 1. 0. -1. 0. ...,1.0
...,...,...
217508,[ 0. -1. -1. -1. 1. 0. 1. 1. 0. 1. 0. -...,0.0
217509,[ 0. -1. -1. -1. 1. 0. 1. 1. 0. 0. 0. -...,0.0
217510,[ 0. 0. -1. -1. 1. 0. 1. 1. 0. 0. 0. -...,0.0
217511,[ 0. 0. -1. -1. 1. 0. 1. 0. 0. 0. 0. -...,0.0


In [14]:
import pickle
#Se prepara y exporta el pickle o resultado del entramiento del juego, se puede ejecutar en visual studio code con un pygame
with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)